In [1]:
import pandas as pd

In [2]:
import numpy as np

# Part 1

protocol://[user[:password]@]hostname/[database_name]


from env import host, user, password

url = f'mysql+pymysql://{user}:{password}@{host}/employees'


In [3]:
import env

In [4]:
from env import host, username, password #importing credentials 

Create function to connect to multiple databases just using function

In [5]:
# def get_db_url(database):
#     return f'mysql+pymysql://{username}:{password}@{host}/{database}'

In [6]:
# env.get_db_url('employees') ##save THIS to variable to use..cool

In [136]:
url = f'mysql+pymysql://{username}:{password}@{host}/employees'

In [8]:
emp = pd.read_sql('select * from employees', url) #save query to df

In [9]:
titles = pd.read_sql('select * from titles', url) #save query to df

In [10]:
type(emp) # verify

pandas.core.frame.DataFrame

In [11]:
type(titles)# verify

pandas.core.frame.DataFrame

In [12]:
emp 

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01
4,10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12
...,...,...,...,...,...,...
300019,499995,1958-09-24,Dekang,Lichtner,F,1993-01-12
300020,499996,1953-03-07,Zito,Baaz,M,1990-09-27
300021,499997,1961-08-03,Berhard,Lenart,M,1986-04-21
300022,499998,1956-09-05,Patricia,Breugel,M,1993-10-13


In [13]:
titles 

,emp_no,title,from_date,to_date
0,10001,Senior Engineer,1986-06-26,9999-01-01
1,10002,Staff,1996-08-03,9999-01-01
2,10003,Senior Engineer,1995-12-03,9999-01-01
3,10004,Engineer,1986-12-01,1995-12-01
4,10004,Senior Engineer,1995-12-01,9999-01-01
...,...,...,...,...
443303,499997,Engineer,1987-08-30,1992-08-29
443304,499997,Senior Engineer,1992-08-29,9999-01-01
443305,499998,Senior Staff,1998-12-27,9999-01-01
443306,499998,Staff,1993-12-27,1998-12-27


In [14]:
emp.shape # rows and columns

(300024, 6)

In [15]:
emp.info() # summary

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300024 entries, 0 to 300023
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   emp_no      300024 non-null  int64 
 1   birth_date  300024 non-null  object
 2   first_name  300024 non-null  object
 3   last_name   300024 non-null  object
 4   gender      300024 non-null  object
 5   hire_date   300024 non-null  object
dtypes: int64(1), object(5)
memory usage: 13.7+ MB


In [16]:
titles.shape

(443308, 4)

In [17]:
titles.info() # summary

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 443308 entries, 0 to 443307
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   emp_no     443308 non-null  int64 
 1   title      443308 non-null  object
 2   from_date  443308 non-null  object
 3   to_date    443308 non-null  object
dtypes: int64(1), object(3)
memory usage: 13.5+ MB


In [18]:
titles.title.nunique(),titles.title.value_counts() # unique titles from sample

(7,
 Engineer              115003
 Staff                 107391
 Senior Engineer        97750
 Senior Staff           92853
 Technique Leader       15159
 Assistant Engineer     15128
 Manager                   24
 Name: title, dtype: int64)

In [19]:
oldest_date = '''
    select min(to_date)
from titles
'''

In [20]:
pd.read_sql(oldest_date, url) #oldest to_date

,min(to_date)
0,1985-03-01


In [21]:
recent_date = '''
select max(to_date)
from titles
where to_date < now()
'''

In [22]:
pd.read_sql(recent_date, url) #most_recent to_date (NOT now)

,max(to_date)
0,2002-08-01


In [23]:
# found the largest date, filter to see NOT equal to that max, wrapping,
# then locating that .max() date not equal to the now date...cool
titles [titles.to_date != titles.to_date.max()].to_date.max()

datetime.date(2002, 8, 1)

# Part 2

## USERS/ROLES

In [24]:
np.random.seed(123)

In [25]:
# Create the users DataFrame.

users = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})
users

,id,name,role_id
0,1,bob,1.0
1,2,joe,2.0
2,3,sally,3.0
3,4,adam,3.0
4,5,jane,NaN
5,6,mike,NaN


In [26]:
# Create the roles DataFrame

roles = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
})
roles

,id,name
0,1,admin
1,2,author
2,3,reviewer
3,4,commenter


What is the result of using a right join on the DataFrames?

In [243]:
pd.merge(users,roles, how='right',left_on='role_id', right_on='id', \
         indicator = True)

,id_x,name_x,role_id,id_y,name_y,_merge
0,1.0,bob,1.0,1,admin,both
1,2.0,joe,2.0,2,author,both
2,3.0,sally,3.0,3,reviewer,both
3,4.0,adam,3.0,3,reviewer,both
4,NaN,NaN,NaN,4,commenter,right_only


In [244]:
# commenter is added with NaN values

What is the result of using an outer join on the DataFrames?

In [29]:
pd.merge(users,roles, how='outer',left_on='role_id', right_on='id', indicator=True)

,id,name,role_id,_merge
0,1,bob,1.0,left_only
1,2,joe,2.0,left_only
2,3,sally,3.0,left_only
3,4,adam,3.0,left_only
4,5,jane,NaN,left_only
5,6,mike,NaN,left_only
6,1,admin,NaN,right_only
7,2,author,NaN,right_only
8,3,reviewer,NaN,right_only
9,4,commenter,NaN,right_only


In [245]:
# Outer join merged all items, creates a messy table, and stacked
# the name columns, keeps everything

What happens if you drop the foreign keys from the DataFrames and try to merge them?

In [248]:
users.drop(columns='role_id').merge(roles, how = 'inner', on='id', indicator = True)

,id,name_x,name_y,_merge
0,1,bob,admin,both
1,2,joe,author,both
2,3,sally,reviewer,both
3,4,adam,commenter,both


In [249]:
# Adam is now a commenter....not correct! We lost the link between tables

## MPG

Load the mpg dataset from PyDataset.

In [86]:
from pydataset import data
mpg = data('mpg')
mpg.head()

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact


Output and read the documentation for the mpg dataset.

In [38]:
data('mpg', show_doc=True) 

mpg

PyDataset Documentation (adopted from R Documentation. The displayed examples are in R)

## Fuel economy data from 1999 and 2008 for 38 popular models of car

### Description

This dataset contains a subset of the fuel economy data that the EPA makes
available on http://fueleconomy.gov. It contains only models which had a new
release every year between 1999 and 2008 - this was used as a proxy for the
popularity of the car.

### Usage

    data(mpg)

### Format

A data frame with 234 rows and 11 variables

### Details

  * manufacturer. 

  * model. 

  * displ. engine displacement, in litres 

  * year. 

  * cyl. number of cylinders 

  * trans. type of transmission 

  * drv. f = front-wheel drive, r = rear wheel drive, 4 = 4wd 

  * cty. city miles per gallon 

  * hwy. highway miles per gallon 

  * fl. 

  * class. 




In [39]:
mpg.shape

(234, 11)

In [40]:
mpg

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact
...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize


Check out your column names and perform any cleanup you may want on them.



In [93]:
mpg.columns = ['manufacturer', 'model', 'displ', 'year', 'cyl',\
                'transmission', 'drive', 'city', 'highway', \
                'type', 'category']

In [94]:
mpg

,manufacturer,model,displ,year,cyl,transmission,drive,city,highway,type,category
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact
...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize


Display the summary statistics for the dataset.

In [253]:
mpg.describe()

,displ,year,cyl,city,highway,mileage_difference,average_mileage
count,234.000000,234.000000,234.000000,234.000000,234.000000,234.000000,234.000000
mean,3.471795,2003.500000,5.888889,16.858974,23.440171,6.581197,20.149573
std,1.291959,4.509646,1.611534,4.255946,5.954643,2.262739,5.050290
min,1.600000,1999.000000,4.000000,9.000000,12.000000,2.000000,10.500000
25%,2.400000,1999.000000,4.000000,14.000000,18.000000,5.000000,15.500000
50%,3.300000,2003.500000,6.000000,17.000000,24.000000,7.000000,20.500000
75%,4.600000,2008.000000,8.000000,19.000000,27.000000,8.000000,23.500000
max,7.000000,2008.000000,8.000000,35.000000,44.000000,12.000000,39.500000


In [251]:
mpg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 234 entries, 1 to 234
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   manufacturer        234 non-null    object 
 1   model               234 non-null    object 
 2   displ               234 non-null    float64
 3   year                234 non-null    int64  
 4   cyl                 234 non-null    int64  
 5   transmission        234 non-null    object 
 6   drive               234 non-null    object 
 7   city                234 non-null    int64  
 8   highway             234 non-null    int64  
 9   type                234 non-null    object 
 10  category            234 non-null    object 
 11  mileage_difference  234 non-null    int64  
 12  average_mileage     234 non-null    float64
 13  is_automatic        234 non-null    bool   
dtypes: bool(1), float64(2), int64(5), object(6)
memory usage: 25.8+ KB


How many different manufacturers are there?



In [97]:
mpg.manufacturer.value_counts(), mpg.manufacturer.nunique()

(dodge         37
 toyota        34
 volkswagen    27
 ford          25
 chevrolet     19
 audi          18
 hyundai       14
 subaru        14
 nissan        13
 honda          9
 jeep           8
 pontiac        5
 land rover     4
 mercury        4
 lincoln        3
 Name: manufacturer, dtype: int64,
 15)

How many different models are there?



In [98]:
mpg.model.value_counts(), mpg.model.nunique()

(caravan 2wd               11
 ram 1500 pickup 4wd       10
 civic                      9
 jetta                      9
 dakota pickup 4wd          9
 mustang                    9
 impreza awd                8
 a4 quattro                 8
 grand cherokee 4wd         8
 a4                         7
 toyota tacoma 4wd          7
 camry solara               7
 camry                      7
 tiburon                    7
 sonata                     7
 passat                     7
 f150 pickup 4wd            7
 durango 4wd                7
 explorer 4wd               6
 new beetle                 6
 altima                     6
 4runner 4wd                6
 forester awd               6
 gti                        5
 c1500 suburban 2wd         5
 corolla                    5
 corvette                   5
 malibu                     5
 grand prix                 5
 k1500 tahoe 4wd            4
 pathfinder 4wd             4
 mountaineer 4wd            4
 range rover                4
 maxima   

Create a column named mileage_difference like you did in the DataFrames exercises; this column should contain the difference between highway and city mileage for each car.



In [99]:
mpg.head()

,manufacturer,model,displ,year,cyl,transmission,drive,city,highway,type,category
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact


In [101]:
mpg['mileage_difference']=(mpg.highway-mpg.city)

In [102]:
mpg.head()

,manufacturer,model,displ,year,cyl,transmission,drive,city,highway,type,category,mileage_difference
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10


Create a column named average_mileage like you did in the DataFrames exercises; this is the mean of the city and highway mileage.



In [255]:
mpg['average_mileage'] = mpg[['city', 'highway']].agg('mean', axis = 1)

In [256]:
mpg.head()

,manufacturer,model,displ,year,cyl,transmission,drive,city,highway,type,category,mileage_difference,average_mileage,is_automatic
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11,23.5,True
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8,25.0,False
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11,25.5,False
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9,25.5,True
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10,21.0,True


Create a new column on the mpg dataset named is_automatic that holds boolean values denoting whether the car has an automatic transmission.



In [112]:
mpg['is_automatic']=(mpg.transmission.str.contains('auto'))

In [113]:
mpg.head()

,manufacturer,model,displ,year,cyl,transmission,drive,city,highway,type,category,mileage_difference,average_mileage,is_automatic
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11,23.5,True
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8,25.0,False
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11,25.5,False
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9,25.5,True
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10,21.0,True


In [ ]:
# mpg.transmission.apply(lambda x: 'auto' in x)
    # in applies to single string
    # the apply allows it to loop through the entire df

Using the mpg dataset, find out which which manufacturer has the best miles per gallon on average?



In [262]:
mpg.groupby('manufacturer').mean().average_mileage.nlargest(1)

manufacturer
honda    28.5
Name: average_mileage, dtype: float64

Do automatic or manual cars have better miles per gallon?



In [265]:
mpg.groupby('is_automatic').mean().average_mileage # manual

is_automatic
False    22.227273
True     19.130573
Name: average_mileage, dtype: float64

In [264]:
# mpg[['manufacturer', 'average_mileage', 'is_automatic']]\
# .sort_values('average_mileage', ascending=False) # manual cars have best

# PART 3

## CHIPOTLE

Use your get_db_url function to help you explore the data from the chipotle database.

In [137]:
url_chipotle = env.get_db_url('chipotle')

In [168]:
cp = pd.read_sql('select * from orders', url_chipotle)

What is the total price for each order?



In [169]:
cp

,id,order_id,quantity,item_name,choice_description,item_price
0,1,1,1,Chips and Fresh Tomato Salsa,nan,$2.39
1,2,1,1,Izze,[Clementine],$3.39
2,3,1,1,Nantucket Nectar,[Apple],$3.39
3,4,1,1,Chips and Tomatillo-Green Chili Salsa,nan,$2.39
4,5,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans, Rice, Cheese, Sour Cream]]",$16.98
...,...,...,...,...,...,...
4617,4618,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Sour Cream, Cheese, Lettuce, Guacamole]]",$11.75
4618,4619,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Sour Cream, Cheese, Lettuce, Guacamole]]",$11.75
4619,4620,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto Beans, Guacamole, Lettuce]]",$11.25
4620,4621,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Lettuce]]",$8.75


In [172]:
cp.item_price

0        2.39 
1        3.39 
2        3.39 
3        2.39 
4       16.98 
         ...  
4617    11.75 
4618    11.75 
4619    11.25 
4620     8.75 
4621     8.75 
Name: item_price, Length: 4622, dtype: object

In [171]:
cp.item_price = cp.item_price.str.replace('$','', regex=True)

In [187]:
cp = cp.astype({'item_price':'float'})

In [189]:
cp

,id,order_id,quantity,item_name,choice_description,item_price
0,1,1,1,Chips and Fresh Tomato Salsa,nan,2.39
1,2,1,1,Izze,[Clementine],3.39
2,3,1,1,Nantucket Nectar,[Apple],3.39
3,4,1,1,Chips and Tomatillo-Green Chili Salsa,nan,2.39
4,5,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans, Rice, Cheese, Sour Cream]]",16.98
...,...,...,...,...,...,...
4617,4618,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Sour Cream, Cheese, Lettuce, Guacamole]]",11.75
4618,4619,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Sour Cream, Cheese, Lettuce, Guacamole]]",11.75
4619,4620,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto Beans, Guacamole, Lettuce]]",11.25
4620,4621,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Lettuce]]",8.75


In [191]:
cp.dtypes

id                      int64
order_id                int64
quantity                int64
item_name              object
choice_description     object
item_price            float64
dtype: object

In [198]:
cp.groupby('order_id').item_price.agg('sum')

order_id
1       11.56
2       16.98
3       12.67
4       21.00
5       13.70
        ...  
1830    23.00
1831    12.90
1832    13.20
1833    23.50
1834    28.75
Name: item_price, Length: 1834, dtype: float64

In [269]:
cp.groupby(['order_id']).sum() # had to change item price from string to a float

,id,quantity,item_price
order_id,,,
1,10,4,11.56
2,5,2,16.98
3,13,2,12.67
4,17,2,21.00
5,21,2,13.70
...,...,...,...
1830,9223,2,23.00
1831,13842,3,12.90
1832,9233,2,13.20


What are the most popular 3 items?



In [200]:
cp.item_name.value_counts().head(3)

Chicken Bowl           726
Chicken Burrito        553
Chips and Guacamole    479
Name: item_name, dtype: int64

Which item has produced the most revenue?



In [203]:
cp.groupby('item_name').item_price.agg('sum').head(1)

item_name
6 Pack Soft Drink    356.95
Name: item_price, dtype: float64

## EMPLOYEES / TITLES

Join the employees and titles DataFrames together.



In [224]:
emp_titles = emp.merge(
                        titles,\
                        how='inner',\
                        on='emp_no',\
                        )

In [225]:
emp_titles

,emp_no,birth_date,first_name,last_name,gender,hire_date,title,from_date,to_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26,Senior Engineer,1986-06-26,9999-01-01
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21,Staff,1996-08-03,9999-01-01
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28,Senior Engineer,1995-12-03,9999-01-01
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,Engineer,1986-12-01,1995-12-01
4,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,Senior Engineer,1995-12-01,9999-01-01
...,...,...,...,...,...,...,...,...,...
443303,499997,1961-08-03,Berhard,Lenart,M,1986-04-21,Engineer,1987-08-30,1992-08-29
443304,499997,1961-08-03,Berhard,Lenart,M,1986-04-21,Senior Engineer,1992-08-29,9999-01-01
443305,499998,1956-09-05,Patricia,Breugel,M,1993-10-13,Senior Staff,1998-12-27,9999-01-01
443306,499998,1956-09-05,Patricia,Breugel,M,1993-10-13,Staff,1993-12-27,1998-12-27


For each title, find the hire date of the employee that was hired most recently with that title.




Write the code necessary to create a cross tabulation of the number of titles by department. (Hint: this will involve a combination of SQL code to pull the necessary data and python/pandas code to perform the manipulations.)

